In [2]:
# First let's create our PySpark instance
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("FD").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
appid = spark._jsc.sc().applicationId()
print("You are working with", cores, "core(s) on appid: ",appid)
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

You are working with 1 core(s) on appid:  application_1742332084551_0002

In [3]:
# Install pandas and numpy in EMR cluster
sc.install_pypi_package("pandas")
sc.install_pypi_package("numpy")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Not uninstalling python-dateutil at /usr/lib/python3.9/site-packages, outside environment /mnt/yarn/usercache/livy/appcache/application_1742332084551_0002/container_1742332084551_0002_01_000001/tmp/spark-2629e653-8d30-4810-84a0-98cb3e05b25a
    Can't uninstall 'python-dateutil'. No files were found to uninstall.



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.17.18 requires python-dateutil<=2.8.2,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.



In [4]:
path = "s3://fd-final/fraud_data/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
fraud_data1_df= spark.read.csv(path+'Fraudulent_E-Commerce_Transaction_Data.csv',
    header=True,
    inferSchema=True,
    multiLine=True,      # Handles multi-line values in a column
    quote='"',           # Ensures quoted strings are treated as single values
    escape='"',          # Handles any extra quotes properly
    sep=",",             # Explicitly specify the separator
    ignoreLeadingWhiteSpace=True,
    ignoreTrailingWhiteSpace=True
)

fraud_data2_df= spark.read.csv(path+'Fraudulent_E-Commerce_Transaction_Data_2.csv',
    header=True,
    inferSchema=True,
    multiLine=True,      # Handles multi-line values in a column
    quote='"',           # Ensures quoted strings are treated as single values
    escape='"',          # Handles any extra quotes properly
    sep=",",             # Explicitly specify the separator
    ignoreLeadingWhiteSpace=True,
    ignoreTrailingWhiteSpace=True
)

fraud_data1_df.show(10, truncate=False)
fraud_data2_df.show(10, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------+------------------------------------+------------------+-------------------+--------------+----------------+--------+------------+-----------------+-----------+---------------+------------------------------------------------------+------------------------------------------------------+-------------+----------------+----------------+
|Transaction ID                      |Customer ID                         |Transaction Amount|Transaction Date   |Payment Method|Product Category|Quantity|Customer Age|Customer Location|Device Used|IP Address     |Shipping Address                                      |Billing Address                                       |Is Fraudulent|Account Age Days|Transaction Hour|
+------------------------------------+------------------------------------+------------------+-------------------+--------------+----------------+--------+------------+-----------------+-----------+---------------+----------------------------------------

In [6]:
#check head of fraud_data1 and fraud_data2 

fraud_data1_df.describe()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, Transaction ID: string, Customer ID: string, Transaction Amount: string, Payment Method: string, Product Category: string, Quantity: string, Customer Age: string, Customer Location: string, Device Used: string, IP Address: string, Shipping Address: string, Billing Address: string, Is Fraudulent: string, Account Age Days: string, Transaction Hour: string]

In [7]:
#check head of fraud_data1 and fraud_data2 

fraud_data2_df.describe()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, Transaction ID: string, Customer ID: string, Transaction Amount: string, Payment Method: string, Product Category: string, Quantity: string, Customer Age: string, Customer Location: string, Device Used: string, IP Address: string, Shipping Address: string, Billing Address: string, Is Fraudulent: string, Account Age Days: string, Transaction Hour: string]

In [8]:
#print schema 

fraud_data1_df.describe()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, Transaction ID: string, Customer ID: string, Transaction Amount: string, Payment Method: string, Product Category: string, Quantity: string, Customer Age: string, Customer Location: string, Device Used: string, IP Address: string, Shipping Address: string, Billing Address: string, Is Fraudulent: string, Account Age Days: string, Transaction Hour: string]

In [9]:
#print schema 

fraud_data2_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Transaction ID: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Transaction Amount: double (nullable = true)
 |-- Transaction Date: timestamp (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Product Category: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Customer Age: integer (nullable = true)
 |-- Customer Location: string (nullable = true)
 |-- Device Used: string (nullable = true)
 |-- IP Address: string (nullable = true)
 |-- Shipping Address: string (nullable = true)
 |-- Billing Address: string (nullable = true)
 |-- Is Fraudulent: integer (nullable = true)
 |-- Account Age Days: integer (nullable = true)
 |-- Transaction Hour: integer (nullable = true)

In [10]:
#join both dataframes together 

fraud_data_df = fraud_data1_df.union(fraud_data2_df)
fraud_data_df.show(10, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------+------------------------------------+------------------+-------------------+--------------+----------------+--------+------------+-----------------+-----------+---------------+------------------------------------------------------+------------------------------------------------------+-------------+----------------+----------------+
|Transaction ID                      |Customer ID                         |Transaction Amount|Transaction Date   |Payment Method|Product Category|Quantity|Customer Age|Customer Location|Device Used|IP Address     |Shipping Address                                      |Billing Address                                       |Is Fraudulent|Account Age Days|Transaction Hour|
+------------------------------------+------------------------------------+------------------+-------------------+--------------+----------------+--------+------------+-----------------+-----------+---------------+----------------------------------------

In [11]:
#check for null values 

from pyspark.sql.functions import col, sum

fraud_data_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in fraud_data_df.columns]).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-----------+------------------+----------------+--------------+----------------+--------+------------+-----------------+-----------+----------+----------------+---------------+-------------+----------------+----------------+
|Transaction ID|Customer ID|Transaction Amount|Transaction Date|Payment Method|Product Category|Quantity|Customer Age|Customer Location|Device Used|IP Address|Shipping Address|Billing Address|Is Fraudulent|Account Age Days|Transaction Hour|
+--------------+-----------+------------------+----------------+--------------+----------------+--------+------------+-----------------+-----------+----------+----------------+---------------+-------------+----------------+----------------+
|             0|          0|                 0|               0|             0|               0|       0|           0|                0|          0|         0|               0|              0|            0|               0|               0|
+--------------+-----------+--------

In [12]:
#Check for duplicates 

fraud_data_df = fraud_data_df.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Aggregations and Transformations 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
#drop unique identifiers, timestamp variable, and shipping address
fraud_data_df = fraud_data_df.drop("Customer ID", "Transaction ID", "IP Address", "Transaction Date", "Shipping Address", "Billing Address", "Customer Location")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
fraud_data_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Transaction Amount: double (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Product Category: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Customer Age: integer (nullable = true)
 |-- Device Used: string (nullable = true)
 |-- label: integer (nullable = true)
 |-- Account Age Days: integer (nullable = true)
 |-- Transaction Hour: integer (nullable = true)

In [16]:
print("Total transactions:", fraud_data_df.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total transactions: 1496586

In [17]:
fraud_data_df.groupBy("Is Fraudulent").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------+
|Is Fraudulent|  count|
+-------------+-------+
|            1|  75060|
|            0|1421526|
+-------------+-------+

In [18]:
fraud_data_df.groupBy("Payment Method").count().orderBy("count", ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+------+
|Payment Method| count|
+--------------+------+
|   credit card|374352|
|    debit card|374229|
| bank transfer|374138|
|        PayPal|373867|
+--------------+------+

In [19]:
from pyspark.sql.functions import avg

fraud_data_df.groupBy("Product Category").agg(avg("Transaction Amount").alias("avg_amount")).orderBy("avg_amount", ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+
|Product Category|        avg_amount|
+----------------+------------------+
|   home & garden|  227.324162121598|
|     electronics|226.95705511228425|
| health & beauty|  226.885974676541|
|    toys & games|226.64777960240286|
|        clothing|226.23190085546685|
+----------------+------------------+

In [20]:
fraud_data_df.groupBy("Device Used", "Is Fraudulent").count().orderBy("count", ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------+------+
|Device Used|Is Fraudulent| count|
+-----------+-------------+------+
|     mobile|            0|474328|
|     tablet|            0|473857|
|    desktop|            0|473341|
|     mobile|            1| 25296|
|     tablet|            1| 24911|
|    desktop|            1| 24853|
+-----------+-------------+------+

In [21]:
fraud_data_df.groupBy("Transaction Hour").count().orderBy("Transaction Hour").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|Transaction Hour|count|
+----------------+-----+
|               0|66540|
|               1|66365|
|               2|66730|
|               3|67091|
|               4|66788|
|               5|66281|
|               6|61557|
|               7|61435|
|               8|60918|
|               9|60568|
|              10|60747|
|              11|60961|
|              12|61266|
|              13|61126|
|              14|60863|
|              15|60745|
|              16|60882|
|              17|60677|
|              18|60890|
|              19|60698|
+----------------+-----+
only showing top 20 rows

In [ ]:
#Vectorization 

In [26]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import * 
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import MinMaxScaler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
# Define categorical and numerical columns
categorical_columns = ["Payment Method", "Product Category", "Device Used"]
numerical_columns = ["Transaction Amount", "Quantity", "Customer Age", "Account Age Days", "Transaction Hour"]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
print("Categorical Columns:", categorical_columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Categorical Columns: ['Payment Method', 'Product Category', 'Device Used']

In [46]:
for col in categorical_columns:
    print(f"Column: {col}, Unique Values: {fraud_data_df.select(col).distinct().count()}")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Column: Payment Method, Unique Values: 4
Column: Product Category, Unique Values: 5
Column: Device Used, Unique Values: 3

In [47]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=col, outputCol=col + "_Index", handleInvalid="keep") for col in categorical_columns]
pipeline = Pipeline(stages=indexers)

# Fit and transform in one step
fraud_data_df = pipeline.fit(fraud_data_df).transform(fraud_data_df)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
# Define final feature columns
feature_columns = numerical_columns + [col + "_Index" for col in categorical_columns]

print("Final Feature Columns:", feature_columns)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Final Feature Columns: ['Transaction Amount', 'Quantity', 'Customer Age', 'Account Age Days', 'Transaction Hour', 'Payment Method_Index', 'Product Category_Index', 'Device Used_Index']

In [50]:
from pyspark.ml.feature import MinMaxScaler

# Normalize feature values to the range [0,1]
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(fraud_data_df)
fraud_data_df = scaler_model.transform(fraud_data_df)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
# Rename target variable to "label"
fraud_data_df = fraud_data_df.withColumnRenamed("Is Fraudulent", "label")

# Select only necessary columns
fraud_data_newdf = fraud_data_df.select("label", "scaled_features")  

# Show new processed dataset
fraud_data_newdf.show(10, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------------------------------------------------------------------------------------------------------------+
|label|scaled_features                                                                                                   |
+-----+------------------------------------------------------------------------------------------------------------------+
|0    |[0.0013268461796048614,0.75,0.47058823529411764,0.9862637362637363,0.13043478260869565,0.6666666666666666,0.0,0.5]|
|0    |[0.011691847066007444,0.25,0.39215686274509803,0.7967032967032968,0.7391304347826086,0.3333333333333333,0.0,0.0]  |
|1    |[0.0507353201883113,0.75,0.43137254901960786,0.07967032967032968,0.8695652173913043,0.3333333333333333,1.0,0.5]   |
|0    |[0.004581716469360017,1.0,0.3529411764705882,0.1098901098901099,0.08695652173913043,1.0,1.0,1.0]                  |
|0    |[0.011393227884255518,1.0,0.6862745098039216,0.2802197802197802,0.9565217391304348,0.3333333333333333,1.0,0.0]    |
|0    |[0.024962

In [55]:
#Write new processed dataset to s3 bucket 

fraud_data_newdf.coalesce(1).write.parquet("s3a://fd-final/fraud_data/fraud_processed.parquet", mode="overwrite")



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…